In [1]:
#https://towardsdatascience.com/efficient-frontier-optimize-portfolio-with-scipy-57456428323e
import pandas as pd
import statsmodels.api as sm
import numpy as np
from scipy.optimize import minimize
import pulp
from function_helper import *

cols = ['Month', 'S/L', 'S/M', 'S/H', 'B/L', 'B/M', 'B/H', 'Russell 3000','T-Bill', 'RMRF', 'Term Spread', 'Yield Spread','change Oil Price']
df = pd.read_csv('Data/hw3.csv',skiprows=1,skipfooter=1,names=cols).dropna()
#Q1
FF_factors = df[['RMRF', 'Term Spread', 'Yield Spread','change Oil Price']]


excess_returns = df[['S/L','S/M','S/H','B/L','B/M','B/H']].subtract(df['T-Bill'],axis=0)
excess_returns.insert(0,'Months',df['Month'])

#Q2
excess_returns_index = df[['S/L','S/M','S/H','B/L','B/M','B/H','Russell 3000']].subtract(df['T-Bill'].dropna(),axis=0)
excess_returns_index.insert(0,'Months',df['Month'])

ols_list = ols(excess_returns,FF_factors)
ols_list_index = ols(excess_returns_index,FF_factors)

Q1
Minimize Portfolio varaince and target at least 12% per year in expected returns but,
1. You would not take oil-price risk
2. You would like your Portfolio to move one to one with the market.

In [44]:
def optimize_1(returns_df,no_securities,risk_factors_df,ols_list, target_return=0.1):

    init_guess = np.array(no_securities*[1./no_securities,])

    bounds = ((-1.0, 1.5),) * no_securities

    weights = minimize(factor_portfolio_variance, init_guess,
                       args=(risk_factors_df,ols_list,), method='SLSQP',
                       options={'disp': False},
                       constraints=({'type': 'eq', 'fun': lambda inputs: np.sum(inputs)-1},
                                    {'type': 'eq', 
                                    'args': (returns_df,),
                                    'fun': lambda inputs, returns_df,:portfolio_return(weights=inputs,returns_df=returns_df)-target_return},
                                    {'type':'eq',
                                    'args':(risk_factors_df,ols_list,),
                                    'fun':lambda inputs,risk_factors_df,ols_list:factor_portfolio_variance(weights=inputs,risk_factors_df=risk_factors_df,ols_list=ols_list)},

                                    {'type':'eq',
                                    'args':(risk_factors_df,ols_list,),
                                    'fun':lambda inputs,risk_factors_df,ols_list:factor_cov(weights=inputs,risk_factors_df=risk_factors_df,ols_list=ols_list,factor_idx=1)},
                                    {'type':'eq',
                                    'args':(risk_factors_df,ols_list,),
                                    'fun':lambda inputs,risk_factors_df,ols_list:factor_cov(weights=inputs,risk_factors_df=risk_factors_df,factor_idx=0,ols_list=ols_list)-0.8}),
                       bounds=bounds)
    return weights.x

opt_weights = optimize_1(returns_df = excess_returns[['S/L','S/M','S/H','B/L','B/M','B/H']],
        no_securities=6,
        risk_factors_df=FF_factors,
        ols_list=ols_list,
        target_return=10/12)

In [45]:
display_answer(FF_factors,ols_list,excess_returns[['S/L','S/M','S/H','B/L','B/M','B/H']],opt_weights)

residual matrix

[10.708987961380439, 5.991602925449216, 8.207421864828664, 1.1009581144375429, 2.6191071457100206, 5.684358088583526]


Portfolio Covariance



Optimized Weights
S/L   -0.593808
S/M    0.108356
S/H   -0.085596
B/L    0.957426
B/M   -0.124630
B/H    0.738252
Name: Weights, dtype: float64


Portfolio Factor Loadings
[ 0.80190508 -0.16208816 -0.53113166 -0.01076262]


Portfolio Excess Return
0.8333333369124174


Portfolio Variance
4.45009691454162


Create Optimal Portfolio tracking Russell 3000 and earning at least 2% per year above expected return on the Russell 3000 by minimizing variance of the tracking errors

In [6]:
def optimize_2(returns_df,no_securities,risk_factors_df,ols_list, target_return=0.1):

    init_guess = np.array(no_securities*[1./no_securities,])

    #bounds = ((-1.5, 1.5),) * no_securities

    weights = minimize(factor_portfolio_variance, init_guess,
                       args=(risk_factors_df,ols_list,),method='SLSQP',
                       options={'disp': False},
                       constraints=({'type': 'eq', 'fun': lambda inputs: inputs[6]+1},
                                    {'type': 'eq', 'fun': lambda inputs: np.sum(inputs)},
                                    {'type': 'eq', 
                                    'args': (returns_df,),
                                    'fun': lambda inputs, returns_df:(portfolio_return(weights=inputs,returns_df = returns_df))-target_return}))
                                    #{'type':'eq',
                                    #'args':(returns_df,),
                                    #'fun':lambda inputs,returns_df:portfolio_variance(weights=inputs,returns_df=returns_df)}))
                       #bounds=bounds)
    return weights.x
opt_weights_2 = optimize_2(returns_df = excess_returns_index[['S/L','S/M','S/H','B/L','B/M','B/H','Russell 3000']],#df[['S/L','S/M','S/H','B/L','B/M','B/H','Russell 3000']],
        no_securities=7,
        risk_factors_df=FF_factors,
        ols_list=ols_list_index,
        target_return=2/12)

In [7]:
display_answer(FF_factors,ols_list_index,excess_returns_index[['S/L','S/M','S/H','B/L','B/M','B/H','Russell 3000']],opt_weights_2)

residual matrix

[10.708987961380439, 5.991602925449216, 8.207421864828664, 1.1009581144375429, 2.6191071457100206, 5.684358088583526, 0.08022564791136581]


Portfolio Covariance



Optimized Weights
S/L            -0.037668
S/M             0.274736
S/H             0.235987
B/L             0.347493
B/M             0.067144
B/H             0.112308
Russell 3000   -1.000000
Name: Weights, dtype: float64


Portfolio Factor Loadings
[-0.00388205  0.05782232  0.11815809 -0.00458802]


Portfolio Excess Return
0.1666666666726447


Portfolio Variance
1.2078982943733179


In [39]:
def optimize_3(returns_df,no_securities,risk_factors_df,ols_list, target_return):

    init_guess = random_weights(no_securities)

    bounds = ((-1.5, 1.5),) * no_securities

    weights = minimize(factor_portfolio_variance, init_guess,
                       args=(risk_factors_df,ols_list,), method='SLSQP',
                       options={'disp': False},
                       constraints=({'type': 'eq', 'fun': lambda inputs: inputs[6]+1},
                                    {'type': 'eq', 'fun': lambda inputs: np.sum(inputs)},
                                    {'type': 'eq', 
                                    'args': (returns_df,),
                                    'fun': lambda inputs, returns_df:portfolio_return(weights=inputs,returns_df=returns_df)-target_return},
                                    {'type':'eq',
                                    'args':(risk_factors_df,ols_list,),
                                    'fun':lambda inputs,risk_factors_df,ols_list:factor_cov(weights=inputs,risk_factors_df=risk_factors_df,ols_list=ols_list,factor_idx=1)},
                                    {'type':'eq',
                                    'args':(risk_factors_df,ols_list,),
                                    'fun':lambda inputs,risk_factors_df,ols_list:factor_cov(weights=inputs,risk_factors_df=risk_factors_df,ols_list=ols_list,factor_idx=0)-0.8}),
                       bounds=bounds)
    return weights.x

In [42]:
opt_weights_3 = optimize_3(returns_df = excess_returns_index[['S/L','S/M','S/H','B/L','B/M','B/H','Russell 3000']],
        no_securities=7,
        risk_factors_df=FF_factors,
        ols_list=ols_list_index,
        target_return=0)

In [46]:
display_answer(FF_factors,ols_list_index,excess_returns_index[['S/L','S/M','S/H','B/L','B/M','B/H','Russell 3000']],opt_weights_3)

residual matrix

[10.708987961380439, 5.991602925449216, 8.207421864828664, 1.1009581144375429, 2.6191071457100206, 5.684358088583526, 0.08022564791136581]


Portfolio Covariance



Optimized Weights
S/L             1.500000
S/M             1.500000
S/H            -0.668327
B/L            -0.151683
B/M            -1.500000
B/H             1.213504
Russell 3000   -0.582633
Name: Weights, dtype: float64


Portfolio Factor Loadings
[ 1.89863693 -0.04914601  0.88155208 -0.02169349]


Portfolio Excess Return
1.0099074741806524


Portfolio Variance
11.27615083144794


In [56]:
excess_returns_4 = pd.concat([df[['S/L','S/M','S/H','B/L','B/M','B/H']],df['Russell 3000']*2],axis=1).subtract(df['T-Bill'].dropna(),axis=0)
excess_returns_4.insert(0,'Months',df['Month'])
ols_list_index_4 = ols(excess_returns_4,FF_factors)

In [60]:
def optimize_4(returns_df,no_securities,risk_factors_df,ols_list, target_return=0.1):

    init_guess = random_weights(no_securities)

    #bounds = ((-1, 1),) * no_securities

    weights = minimize(factor_portfolio_variance, init_guess,
                       args=(risk_factors_df,ols_list,), method='SLSQP',
                       options={'disp': False},
                       constraints=({'type': 'eq', 'fun': lambda inputs: inputs[6]+1},
                                    {'type': 'eq', 'fun': lambda inputs: np.sum(inputs)}))
                                    #{'type': 'eq', 
                                    #'args': (returns_df,),
                                    #'fun': lambda inputs, returns_df:portfolio_return(weights=inputs,returns_df=returns_df)-target_return}))
                                    #{'type':'eq',
                                    #'args':(risk_factors_df,ols_list,),
                                    #'fun':lambda inputs,risk_factors_df,ols_list:factor_cov(weights=inputs,risk_factors_df=risk_factors_df,ols_list=ols_list,factor_idx=0)},
                                    #{'type':'eq',
                                    #'args':(risk_factors_df,ols_list,),
                                    #'fun':lambda inputs,risk_factors_df,ols_list:factor_cov(weights=inputs,risk_factors_df=risk_factors_df,ols_list=ols_list,factor_idx=3)}))
                       #bounds=bounds)
    return weights.x

In [61]:
opt_weights_4 = optimize_4(returns_df = excess_returns_4[['S/L','S/M','S/H','B/L','B/M','B/H','Russell 3000']],
        no_securities=7,
        risk_factors_df=FF_factors,
        ols_list=ols_list_index_4,
        target_return=-12.04/12)

In [62]:
display_answer(FF_factors,ols_list_index_4,excess_returns_4[['S/L','S/M','S/H','B/L','B/M','B/H','Russell 3000']],opt_weights_4)

residual matrix

[10.708987961380439, 5.991602925449216, 8.207421864828664, 1.1009581144375429, 2.6191071457100206, 5.684358088583526, 0.3708015916496651]


Portfolio Covariance

                    S/L        S/M        S/H        B/L        B/M  \
S/L           43.713678  25.980713  25.409368  25.292570  22.926939   
S/M           25.980713  26.459981  20.021424  19.886148  18.043647   
S/H           25.409368  20.021424  27.820944  19.459729  17.669747   
B/L           25.292570  19.886148  19.459729  20.525508  17.584392   
B/M           22.926939  18.043647  17.669747  17.584392  18.605216   
B/H           24.003947  18.866188  18.476979  18.448381  16.757651   
Russell 3000  49.891217  39.238756  38.377699  38.281898  34.725043   

                    B/H  Russell 3000  
S/L           24.003947     49.891217  
S/M           18.866188     39.238756  
S/H           18.476979     38.377699  
B/L           18.448381     38.281898  
B/M           16.757651     34.725043  
B/H         